In [0]:
# 1 
# Importing all the required libraries 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

In [0]:
# Configuring Google colab to pick the input files from Google Drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 4.7MB/s 


In [0]:
# Get the Stemmed Text CSV file
link = "https://drive.google.com/open?id=10ZYvHgQUs7YD0mmSy4jow0SZYWO-6r9l" # link to google drive Stemmed Text.csv
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
_stem_text_df = pd.read_csv('Filename.csv')
# Remove the files without any Eglish text (feature vector)
_stem_text_df["English"] = _stem_text_df["Text"].str.contains('english version document avail sinc includ english special edit', regex=True)
#_stem_text_df['English'].value_counts()

In [0]:
# Get the Eurovoc Labels from Google Drive
link = "https://drive.google.com/open?id=1l_LgY8OKrW4tCuCg17kI-6c0qN1ZiYBp" # link to google drive Stemmed Text.csv
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
_label_df = pd.read_csv("Filename.csv")    

Insert Graphs from Rabi's code here

In [0]:
# Reading from labels file and transposing them. One row each for a file. 
# Transpose using pivot table. So, we have as many columns as distinct labels. Value under a label shows whether the document has the label or not.
_label_df['pivotcol'] = 1
_lbl_trnsps_df = pd.pivot_table(_label_df,values='pivotcol',index=['Filename'],columns=['Eurovoc_Label'],aggfunc=np.sum, fill_value=0)

In [0]:
# Get the list of labels into a column
_label_df = _label_df.groupby(['Filename'])['Eurovoc_Label'].apply(list).reset_index()

In [0]:
# merge the features and labels transposed 
df_for_stemming = _stem_text_df[_stem_text_df['English']==False]
_stem_text_df = df_for_stemming.drop(columns='English')
_array_df = pd.merge(_stem_text_df, _lbl_trnsps_df, on='Filename')
_array_df = _array_df.drop(columns='Filename')

In [0]:
Y_labels = pd.merge(_stem_text_df, _label_df, on='Filename')
Y_labels = Y_labels.drop(columns = ["Text","Filename"])

In [0]:
Y_labels

,Eurovoc_Label
0,"[animal plague, swine, region, Austria, wild m..."
1,"[barley, intervention agency, Hungary, award o..."
2,"[common agricultural policy, common organisati..."
3,"[EAGGF Guarantee Section, closing of accounts,..."
4,"[concessionnaire, fine, infringement of Commun..."
5,"[EC agreement, Norway, information transfer, d..."
6,"[administrative transparency, monitoring of ex..."
7,"[fresh water, fish, product quality, environme..."
8,"[vineyard, plant propagation, seed, seedling, ..."
9,"[Switzerland, Liechtenstein, ECSC, internation..."


In [0]:
corr = _array_df.drop(columns = "Text")
# checking the correlation between the labels
corr = corr.corr()

In [0]:
#### for dropping the correlated labels 

col = corr.columns
cols_to_drop = []
k =0
num_classes = 0
for i in range(0,len(corr)):
  for j in range(0,len(corr)):
   # try: 
      if((float(corr[col[i]][j])>.9) and  (col[j] != i) and i<j):
        if(sum(_array_df[col[i]])) < sum(_array_df[col[j]]):
          cols_to_drop.append(col[i])
        else:
          cols_to_drop.append(col[j])
        
      #print(corr[i][j])
      #print("feture1",i,"feature 2",col[j])
        num_classes = num_classes +1
        print(num_classes)
    #except:
     # print(_array_df[col[i]],_array_df[col[j]])
     # continue

# we have 128 labels ( .9 corr)which are highly correlated. possibly due to subcategories. Hence learning subcategory is same as learning the main class. 
# we have 183 labels ( .8 corr)
# we havev 153 labels which have just 1 dataset and they co-occur. 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


In [0]:
cols_to_drop

['Fisheries',
 'Animal health and zootechnics',
 'Approximation of laws and health measures',
 'Approximation of laws and health measures',
 'Customs Union and free movement of goods',
 'Individual tax exemptions',
 'Market access',
 'Market operation',
 'Reliefs from duty',
 'Shipping',
 'Victualling procedures',
 'Euro-Arab cooperation',
 'General provisions',
 'Judicial cooperation in civil matters',
 'Dominica',
 'cost-benefit analysis',
 'cooperation procedure',
 'Corporation tax',
 'Company law',
 'Elimination of double taxation',
 'Corporation tax',
 'Elimination of double taxation',
 'disaffection of young people',
 'Individual tax exemptions',
 'Market access',
 'Market operation',
 'Reliefs from duty',
 'Shipping',
 'Victualling procedures',
 'Elimination of double taxation',
 'legal hearing',
 'constitutional law',
 'Nuclear energy',
 'Environment, consumers and health protection',
 'Ethiopia',
 'education budget',
 'national election',
 'Lomé Convention',
 'Judicial coopera

In [0]:
_array_df = _array_df.drop(columns = cols_to_drop)

In [0]:
### List drops

for j in cols_to_drop:
  for i in Y_labels["Eurovoc_Label"]:
  
    if(j in i):
      print(i)
      print("removing",j)
      i.remove(j)
      print(i)
Y = Y_labels

['Fisheries', 'External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
removing Fisheries
['External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
['Fisheries', 'External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
removing Fisheries
['External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
['Agriculture', 'Approximation of laws and health measures', 'Animal health and zootechnics']
removing Animal health and zootechnics
['Agriculture', 'Approximation of laws 

In [0]:
Y.head()

,Eurovoc_Label
0,"[animal plague, swine, region, Austria, wild m..."
1,"[barley, intervention agency, Hungary, award o..."
2,"[common agricultural policy, common organisati..."
3,"[EAGGF Guarantee Section, closing of accounts,..."
4,"[concessionnaire, fine, infringement of Commun..."


In [0]:
import gc
gc.collect()

233

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
#multilabel_binarizer = MultiLabelBinarizer()
#multilabel_binarizer.fit(Y.Eurovoc_Label)
#Y = multilabel_binarizer.transform(Y.Eurovoc_Label)

count_vect = CountVectorizer(max_df = .9,min_df = .001,token_pattern='[a-zA-Z]+',stop_words="english")
X_counts = count_vect.fit_transform(_array_df.Text)

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
lsa = TruncatedSVD(n_components = 10000)
dtm_lsa = lsa.fit_transform(X_tfidf)

In [0]:
### comment this if previous cell is not commented

from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(Y.Eurovoc_Label)
Y = multilabel_binarizer.transform(Y.Eurovoc_Label)

In [0]:
#confirming the shape of the array

Y.shape

(19399, 3926)

In [0]:
import scipy.sparse as sparse
X_tfidf = sparse.csr_matrix(dtm_lsa)


session crashes

In [0]:
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler


# ros = RandomUnderSampler("majority")
# X_tfidf_resampled, Y_tfidf_resampled = ros.fit_sample(X_tfidf, Y)

In [0]:
from sklearn.model_selection import train_test_split

# time before starting
#removing resampled....... 
#x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf_resampled, Y_tfidf_resampled, test_size=0.2, random_state=71)

x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, Y, test_size=0.2, random_state=71)
# time after finishing
import gc
gc.collect()

1122

Building unbalanced Linear SVM

In [0]:
# for Unbalanced
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(penalty = "l2",loss='hinge', alpha=1e-3, random_state=42, max_iter=50, tol=None)
clf = OneVsRestClassifier(sgd)
model = clf.fit(x_train_tfidf, y_train_tfidf)
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
y_pred = clf.predict(x_test_tfidf)

In [0]:

from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
y_pred = clf.predict(x_test_tfidf)

print("Hamming Loss Hammingloss",hamming_loss(y_pred,y_test_tfidf))
print("f1_score Micro UnBalanced ",f1_score(y_pred, y_test_tfidf,average='micro'))

print("f1_score Macro UnBalanced",f1_score(y_pred, y_test_tfidf,average='macro'))



Building balanced Linear SVM

In [0]:
# for Building unbalanced Linear SVM
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(penalty = "l2",loss='hinge', alpha=1e-3, random_state=42, max_iter=50, tol=None,class_weight = "balanced")
clf = OneVsRestClassifier(sgd)
model = clf.fit(x_train_tfidf, y_train_tfidf)
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
y_pred = clf.predict(x_test_tfidf)

print("Hamming Loss Hammingloss",hamming_loss(y_pred,y_test_tfidf))
print("f1_score Micro Balanced ",f1_score(y_pred, y_test_tfidf,average='micro'))

print("f1_score Macro Balanced",f1_score(y_pred, y_test_tfidf,average='macro'))